# 環境建置及文件下載

In [ ]:
import os

# 從雲端下載檔案的程式
def doc_download(Dataset_File,Data_url):
    """
    雲端檔案讀入
    """
    if not os.path.isfile(Dataset_File):
      url = url_trans(Data_url)
      os.system(f"wget {url} -O {Dataset_File}")

def url_trans(url):
    """
    將 Google Drive 網址轉換
    """
    if "https://drive.google.com/file/d/" in url:
        # 提取 ID 部分
        file_id = url.split("/file/d/")[1].split("/")[0]
        # 拼接成新的下載鏈接
        direct_url = f"https://drive.google.com/uc?id={file_id}"
        return direct_url
    else:
        raise ValueError("提供的網址格式不正確。請確認是 Google Drive 分享網址。")

In [ ]:
# 使用者名稱
user_name = "郭大呈"


In [ ]:
# 下載訊息資料
doc_download("sample_5000_messages.csv","https://drive.google.com/file/d/1ZtkdbXzNfYxq94G6DDBSv7Ul7uS1oHsO/view?usp=drive_link")

# 下載專有名詞資料
doc_download("new_word.txt","https://drive.google.com/file/d/1R-GPI-XPUeysKl49o3VkDUiqGnS0zZcK/view?usp=drive_link")

# 下載斷詞資料
doc_download("Chinese_stopwords.txt","https://drive.google.com/file/d/1k5DfcWeyZCcLhPrcJnxGXYJrrVcszKUV/view?usp=drive_link")

# 下載label資料
doc_download("rated.csv","https://drive.google.com/file/d/1rzF0BtKy8hocyBJ9uR_pALeupppdzApu/view?usp=drive_link")

說明：


1.   因為個資問題，sample_5000_messages是隨機從我的對話紀錄裡抽取5000筆訊息，所以模型實際運行的結果可能效能會受影響
2.   new_word.txt是根據助教課給的範例在新增一些instagram文本有的專有名詞，比如"發起電話"等



# 資料儲存成df

In [ ]:
import pandas as pd

In [ ]:
file_path = "sample_5000_messages.csv"
df = pd.read_csv(file_path)
df

,聊天室名稱,訊息傳送者,訊息內容,傳送時間
0,:0,郭大呈,70,2023/10/18 12:05
1,__.nywwc,郭大呈,打爆,2024/11/1 13:21
2,__.nywwc,__.nywwc,^_^,2023/5/26 14:47
3,__.nywwc,郭大呈,好ㄚ,2023/10/17 14:28
4,__.nywwc,__.nywwc,誰是嘟嘟麵,2024/7/21 04:31
...,...,...,...,...
4995,攝氏温度,攝氏温度,謝謝學長😍,2024/7/19 10:15
4996,攝氏温度,郭大呈,謝謝yellow friend,2024/9/16 15:34
4997,顧寬証,郭大呈,我們之前都打通宵場打完吃早餐😵‍💫,2023/7/17 12:19
4998,顧寬証,郭大呈,111306077,2023/9/18 16:10


In [ ]:
# 將評等資料加入訊息欄
rated = pd.read_csv('rated.csv')
df = df.merge(rated, left_on='訊息傳送者', right_on='聯絡人', how='left')

# 填充缺失值為 1
df['評等'] = df['評等'].fillna(1)

# 如果不需要 '聯絡人' 欄位，可以將其刪除
df = df.drop(columns=['聯絡人'])
df

,聊天室名稱,訊息傳送者,訊息內容,傳送時間,評等
0,:0,郭大呈,70,2023/10/18 12:05,1.0
1,__.nywwc,郭大呈,打爆,2024/11/1 13:21,1.0
2,__.nywwc,__.nywwc,^_^,2023/5/26 14:47,5.0
3,__.nywwc,郭大呈,好ㄚ,2023/10/17 14:28,1.0
4,__.nywwc,__.nywwc,誰是嘟嘟麵,2024/7/21 04:31,5.0
...,...,...,...,...,...
4995,攝氏温度,攝氏温度,謝謝學長😍,2024/7/19 10:15,4.0
4996,攝氏温度,郭大呈,謝謝yellow friend,2024/9/16 15:34,1.0
4997,顧寬証,郭大呈,我們之前都打通宵場打完吃早餐😵‍💫,2023/7/17 12:19,1.0
4998,顧寬証,郭大呈,111306077,2023/9/18 16:10,1.0


# 斷詞

1️⃣ import 需要的套件

如jieba（結巴）、wordcloud（文字雲）、matplotlib等作圖套件。

In [ ]:
!pip install jieba
!pip install Pillow
!pip install spacy
!python -m spacy download zh_core_web_sm

import jieba
import spacy
from os import path
from wordcloud import WordCloud
from PIL import Image
import matplotlib.pyplot as plt
from wordcloud import ImageColorGenerator
import numpy as np
import re

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 20.8 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('zh_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


2️⃣ 資料清洗：

*   專有名詞字典：專有名詞要整句識別，不可以被斷詞（需要整句拿掉）。
*   訊息內容：類型確認（字串）和空值修正（空字串）。
*   去除停詞：標點符號（如「」？『』！，）、連接詞（如若、罷了、假如）、語助詞（啊、哎、吧、吧噠）等等。
*   需移除的文字包含英文、數字的空字串和空字串。

In [ ]:
#  建立字典，幫助工具判斷專有名詞
new_word = open("new_word.txt", "r", encoding='UTF-8').read()
jieba.load_userdict("new_word.txt")

# 確保訊息內容為字串類型，非字串值替換為空字串
df['訊息內容'] = df['訊息內容'].astype(str)

#初步斷詞
df['訊息斷詞'] = df['訊息內容'].apply(lambda message: jieba.lcut(message))

#去除停詞
stopword = open("Chinese_stopwords.txt", "r", encoding='UTF-8').read()
stopword_list = stopword.split("\n")
df['訊息斷詞'] = df['訊息斷詞'].apply(lambda tokens: [token for token in tokens if token not in stopword_list])

# 移除包含英文和數字的字串及空字串
df['訊息斷詞'] = df['訊息斷詞'].apply(
    lambda tokens: [word for word in tokens if not re.search(r'[a-zA-Z0-9]', word) and word.strip()]
)

# 顯示結果
print(df[['訊息內容', '訊息斷詞']].head(30))

Building prefix dict from the default dictionary ...
DEBUG:jieba:Building prefix dict from the default dictionary ...
Dumping model to file cache /tmp/jieba.cache
DEBUG:jieba:Dumping model to file cache /tmp/jieba.cache
Loading model cost 1.147 seconds.
DEBUG:jieba:Loading model cost 1.147 seconds.
Prefix dict has been built successfully.
DEBUG:jieba:Prefix dict has been built successfully.


                        訊息內容                    訊息斷詞
0                         70                      []
1                         打爆                    [打爆]
2                        ^_^                      []
3                         好ㄚ                  [好, ㄚ]
4                      誰是嘟嘟麵                 [嘟嘟, 麵]
5             還是看你們想去哪裡ㄧㄝㄎㄧㄥ  [想, 哪裡, ㄧ, ㄝ, ㄎ, ㄧ, ㄥ]
6                        沒笑死                    [笑死]
7                   他之前中午有開ㄉ         [之前, 中午, 有開, ㄉ]
8                         ㄏㄚ                  [ㄏ, ㄚ]
9                        nan                      []
10                        下次                    [下次]
11                        靠北                    [靠北]
12                    我截圖給他看                [我截, 圖給]
13                   我以為他去按你                      []
14                    就我都不會寫                     [寫]
15                把ㄅㄨ看進來的正前方             [ㄅ, ㄨ, 正前方]
16                     我以為是你                      []
17                 欸你為啥認識鄭凱穎            [欸, 認識

3️⃣ 資料彙整：

*   只保留對方傳送的訊息，過濾掉自己傳送給對方的訊息（因為發言者不同會產生不一樣的詞彙，例如對方可能會傳「郭大呈」或「大呈」但本人不會打「郭大呈」）。
*   一年的對話紀錄拆成12個月份，增加資料樣本數量提升準確度。
*   按照訊息轉送者和月份分組，方便彙整資料樣本。



In [ ]:
# 過濾非 "使用者" 的訊息
df_filtered = df[df["訊息傳送者"] != user_name].copy()

# 處理傳送時間，提取年份與月份
df_filtered['傳送時間'] = pd.to_datetime(df_filtered['傳送時間'], errors='coerce')
df_filtered['年'] = df_filtered['傳送時間'].dt.strftime('%Y')
df_filtered['月份'] = df_filtered['傳送時間'].dt.strftime('%m')

# 過濾年份為 2024 的訊息
df_filtered = df_filtered[df_filtered['年'] == "2024"].copy()

# 按 "訊息傳送者" 和 "月份" 分組，彙整結果
grouped = df_filtered.groupby(["訊息傳送者", "月份"]).agg(
    總斷詞結果=("訊息斷詞", lambda x: [word for sublist in x for word in sublist]),
    評等=("評等", "first")
).reset_index()

print(grouped)



         訊息傳送者  月份                                              總斷詞結果   評等
0    Anson(けん)  03                                           [時間, 社長]  3.0
1    Anson(けん)  04                                          [攝影社, 怪人]  3.0
2    Anson(けん)  08                                   [研究所, 我現, 不想, 讀]  3.0
3         Cleo  09                            [偏, 簡單, 粗暴, 之前, 想, 恐怖片]  4.0
4         Cleo  10  [旋轉, 木馬, 確實, 一部, 跳舞, 跳, 好, 看看, 會不會, 好, 中, 羹, 魯...  4.0
..         ...  ..                                                ...  ...
324        黃聖温  09                                       [._., 起床, 🥹]  5.0
325        黃雋恆  08                                               [肯定]  4.0
326        黃雋恆  09                                              [真羨慕]  4.0
327        黃雋恆  10                             [就要, 出發, 喔, 好像, 唱, 煎熬]  4.0
328   𝙍𝙞𝙘𝙠𝙮 𝙇𝙤  09                                    [朋友, 貌似, 急事, 找]  2.0

[329 rows x 4 columns]


# TF-IDF

1️⃣ 尋找資料特徵

*   引入 TF-IDF 套件。
*   利用 TF-IDF 計算文檔中的詞頻和逆文檔頻率，並用 tfidf_matrix 儲存矩陣。
*   最後用 dataframe 顯示結果。



In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
grouped['斷詞訊息_str'] = grouped['總斷詞結果'].apply(lambda x: ' '.join(x))

# 建立 TfidfVectorizer
vectorizer = TfidfVectorizer()

# 計算 TF-IDF 特徵
tfidf_matrix = vectorizer.fit_transform(grouped['斷詞訊息_str'])

# 將 TF-IDF 特徵轉換為 DataFrame
tfidf_features = pd.DataFrame(
    tfidf_matrix.toarray(),
    columns=vectorizer.get_feature_names_out()
)



# 顯示結果
print(tfidf_features.head(20))

     __   一下   一些   一個   一堆   一天  一學分        一定   一幫   一年  ...  館之類   香菇   騎腳  \
0   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  ...  0.0  0.0  0.0   
1   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  ...  0.0  0.0  0.0   
2   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  ...  0.0  0.0  0.0   
3   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  ...  0.0  0.0  0.0   
4   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  ...  0.0  0.0  0.0   
5   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  ...  0.0  0.0  0.0   
6   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  ...  0.0  0.0  0.0   
7   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  ...  0.0  0.0  0.0   
8   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  ...  0.0  0.0  0.0   
9   0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  ...  0.0  0.0  0.0   
10  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.000000  0.0  0.0  ...  0.0  0.0  0.0   
11  0.0  0.0  0.0  0.0  0.0 

# 模型訓練

1️⃣ 資料前處理
*   將「評等」分成五個類別，標籤為 0（非常不熟）、1（不熟）、2（普通）、3（熟）、4（非常熟）。

*   類別資料需要用 onehot-encoder 轉換為二進位表示法。
*   把計算出來的特徵設為 X（自變數），標籤（二進位評等）設為 Y（應變數）並切分資料集（80%）與測試集（20%）。
*   隨機森林模型需要單一類別標籤，所以把 onehot-encoding 後的二進位表示還原回標籤。

2️⃣ 隨機森林模型
*   數的數量設為 100，並利用測試集進行預測。

3️⃣ 模型評估
*   使用 accuracy_score 計算預測準確率。
*   使用 classification_report 詳細報告分類績效。

*備註，因抽取過後隨機訊息評分為"1"和"2"的文本極少(如果你跟這個熟你本來就不太會跟他講話)，所以最後訓練模型只會分出三類



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
import numpy as np

# 假設將連續的 '評等' 分成 5 個類別
num_classes = 4
y_class = pd.cut(grouped['評等'], bins=num_classes, labels=False)  # 分箱處理，標籤為 0, 1, ..., 4

# 將類別轉為 One-Hot
onehot_encoder = OneHotEncoder(sparse_output=False)
y_onehot = onehot_encoder.fit_transform(y_class.values.reshape(-1, 1))

# 資料集、測試集切分
X = tfidf_features
y = y_onehot
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 訓練資料需要將 One-Hot encoding 還原為標籤（分類編號）
y_train_labels = np.argmax(y_train, axis=1)
y_test_labels = np.argmax(y_test, axis=1)

# 初始化並訓練隨機森林（rf）模型
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train_labels)

# 預測
y_pred = rf_model.predict(X_test)

# 評估
accuracy = accuracy_score(y_test_labels, y_pred)
print(f"Accuracy: {accuracy}")
print("\nClassification Report:\n", classification_report(y_test_labels, y_pred))



Accuracy: 0.5757575757575758

Classification Report:
               precision    recall  f1-score   support

           1       0.00      0.00      0.00        10
           2       0.00      0.00      0.00        16
           3       0.59      0.95      0.73        40

    accuracy                           0.58        66
   macro avg       0.20      0.32      0.24        66
weighted avg       0.36      0.58      0.44        66



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


# 預測結果


1️⃣ 查看預測結果：將測試資料（y_test_labels）和預測結果（y_pred）放在 dataframe 中進行對比。

In [ ]:
labels_comparison = pd.DataFrame({
    "y_test_labels": y_test_labels,
    "y_pred": y_pred
})

labels_comparison

,y_test_labels,y_pred
0,1,3
1,2,3
2,2,3
3,3,3
4,3,3
...,...,...
61,3,3
62,3,3
63,1,3
64,1,3
